In [1]:
import os

In [2]:
%pwd

'd:\\End-to-end-Default-Risk-Pred-mlProject\\notebook'

In [3]:
# Change the current working directory to the project root
os.chdir("../")

In [4]:
%pwd

'd:\\End-to-end-Default-Risk-Pred-mlProject'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_data_file: Path
    local_data_file: Path

In [6]:
from defaultMlProj.constants.constant import *
from defaultMlProj.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_data_file=config.source_data_file,
            local_data_file=config.local_data_file
        )
        return data_ingestion_config

In [8]:
import os
import shutil
from defaultMlProj import logger
from defaultMlProj.utils.common import get_size


In [13]:
class DataIngestion:
    def __init__ (self, config: DataIngestionConfig):
        self.config = config

    def copy_data_file(self):

        source = Path(self.config.source_data_file)
        destination = Path(self.config.local_data_file)

        try:
            logger.info(f"Starting data ingestion:copying{source} to {destination}")

            destination.parent.mkdir(parents=True, exist_ok=True)

            if not source.exists():
                raise Exception(f"Source file {source.absolute()} does not exist")

            if destination.exists():
                logger.info(f"File destination {destination} already exists. Skipping copy.")
            else:
                shutil.copy(source, destination)
                logger.info(f"File copied successfully: {source} to {destination}")

        except Exception as e:
            logger.exception(f"Error occurred while copying data file: {e}")
            raise e

In [12]:
from pathlib import Path

# Define the expected path
source_path = Path("notebook/data/default.csv")

print("Current working directory:", Path(".").absolute())
print("Expected source path:", source_path.absolute())
print("Does file exist?", source_path.exists())

Current working directory: d:\End-to-end-Default-Risk-Pred-mlProject
Expected source path: d:\End-to-end-Default-Risk-Pred-mlProject\notebook\data\default.csv
Does file exist? True


In [14]:
# Updating the pipeline item on the workflow list
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.copy_data_file()
except Exception as e:
    raise e

[2025-07-22 23:44:21,641: INFO: common: YAML file config\config.yaml loaded successfully.]
[2025-07-22 23:44:21,645: INFO: common: YAML file params.yaml loaded successfully.]
[2025-07-22 23:44:21,647: INFO: common: Created directory: artifacts]
[2025-07-22 23:44:21,648: INFO: common: Created directory: artifacts/data_ingestion]
[2025-07-22 23:44:21,649: INFO: 653153596: Starting data ingestion:copyingnotebook\data\default.csv to artifacts\data_ingestion\default.csv]
[2025-07-22 23:44:21,652: INFO: 653153596: File copied successfully: notebook\data\default.csv to artifacts\data_ingestion\default.csv]
